# Identifying Hate Speech in Social Media Comments: PyTorch LSTM

### Import Modules

In [0]:
import os
import re
import sys
import ssl
import json
import nltk
import time
import math
import string
import pickle
import argparse

import torch
import numpy as np
import pandas as pd
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from collections import Counter, OrderedDict
from sklearn.model_selection import train_test_split
from torchtext.data import Field, BucketIterator
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pad_packed_sequence, pack_padded_sequence

In [0]:
import warnings
# warnings.filterwarnings('ignore')

warnings.filterwarnings(action='once')

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import files, auth
auth.authenticate_user()
uploaded = files.upload()

/usr/local/lib/python3.6/dist-packages/google/colab/auth.py:158: ResourceWarning: unclosed <ssl.SSLSocket fd=56, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 47294), raddr=('108.177.121.84', 443)>
  if _check_adc():


Saving full_list.json to full_list.json


In [0]:
import exec_lstm as exl

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

In [0]:
torch.cuda.is_available()

True

In [0]:
# import importlib
# import google
# importlib.reload(google.colab.auth)

In [0]:
# from google.colab import auth
# auth.authenticate_user()

/usr/local/lib/python3.6/dist-packages/google/colab/auth.py:141: ResourceWarning: unclosed <ssl.SSLSocket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 56462), raddr=('209.85.145.84', 443)>
  if _check_adc():


In [0]:
!pip install gcsfs

     |████████████████████████████████| 61kB 24.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/9f/0f/b9/5bc5222756d121ccace51ab3084a1c733380908a4e2f939038
Successfully built gcsfs


/usr/local/lib/python3.6/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.6/dist-packages/gcsfs-0.2.2.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


In [0]:
import gcsfs
train_full = pd.read_csv('gs://no-hate/train.csv')

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [0]:
train_full['label'] = train_full.target.apply(lambda x: 1 if x > 0.5 else 0)

In [0]:
test_ratio = 0.2

## Housekeeping and Variable Storage

In [0]:
torch.cuda.empty_cache()

In [0]:
project_id = 'tribal-monolith-242216'
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud alpha survey



In [0]:
def send_to_bucket(filename, bucket_name="no-hate", directory=None):
    if directory:
        !gsutil cp /content/{filename} gs://{bucket_name}/{directory}/
    else:
        !gsutil cp /content/{filename} gs://{bucket_name}/
        
    

In [0]:
# send_to_bucket(filename='20pct_imbalanced_model_state.pt')

In [0]:
import io
# import cloudstorage
import gcsfs
from io import BytesIO
import pickle

def pickle_to_gcs(df, filename, bucket_name="no-hate", directory=None):
    pickle_buffer = io.BytesIO()
    fs = gcsfs.GCSFileSystem(project=project_id)
    df.to_pickle(filename)
    
    if directory:
        directory = directory + "/"
    else:
        directory = ""
    
    with fs.open(f"{bucket_name}/{directory}{filename}", "wb") as handle:    
        pickle.dump(df, handle)
        
    
def load_pickle_from_gcs(filename, bucket_name="no-hate",  directory=None):
    fs = gcsfs.GCSFileSystem(project=project_id) 
    
    if directory:
        directory = directory + "/"
    else:
        directory = ""
        
    with fs.open(f"{bucket_name}/{directory}{filename}", "rb") as handle:
        df = pickle.load(handle)
    return df

In [0]:
# pickle_to_gcs(test_set, '20pct_test_set.pkl')

In [0]:
# print(test_set.shape, test_set.comment_text.isna().sum())

In [0]:
# check = load_pickle_from_gcs('20pct_test_set.pkl')

In [0]:
# print(check.shape, check.comment_text.isna().sum())

In [0]:
# Check Available RAM

# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

In [0]:
# Clear RAM

# !kill -9 -1

## Creating Batch Loader to Enable Larger Dataset Training

Use 20% data partition while building batch loader for LSTM model.

### Create Batch Loader Test Data

In [0]:
# train_full = pd.read_csv("train.csv")

In [0]:
train_set, val_set, test_set = exl.get_samples(train_full, proportion=0.2, 
                                                  train_test_ratio=(1-test_ratio))

In [0]:
print(train_sample.shape, val_set.shape, test_set.shape)

In [0]:
train_sample.loc[train_sample.comment_text != ''].shape

In [0]:
train_sample.head()

In [0]:
# train_drop = train_sample[train_sample.comment_text.apply(exl.tokenizer).apply(len) <= 0].index
# train_drop

In [0]:
# train_sample = train_sample.drop(train_drop, axis = 0)

In [0]:
val_samples = val_set.sample( n=math.ceil(len(train_sample)*test_ratio), random_state=1008 )

In [0]:
val_samples.shape

In [0]:
# val_drop = val_samples[val_samples.comment_text.apply(exl.tokenizer).apply(len) <= 0].index
# val_drop

In [0]:
# val_samples = val_samples.drop(val_drop, axis = 0)

In [0]:
test_samples = test_set.sample( n=math.ceil(len(train_sample)*test_ratio), random_state=1008 )

In [0]:
# test_drop = test_samples[test_samples.comment_text.apply(exl.tokenizer).apply(len) <= 0].index
# test_drop

In [0]:
# test_samples = test_samples.drop(test_drop, axis = 0)

### Data Loader Class

In [0]:
# H/T for the torchtext dataframe-based-Dataset hack to: 
# https://stackoverflow.com/questions/52602071/dataframe-as-datasource-in-torchtext
from torchtext.data import Field, Dataset, Example
import pandas as pd

class DataFrameDataset(Dataset):
    """Class for using pandas DataFrames as a datasource"""
    def __init__(self, examples, fields, filter_pred=None):
        """
        Create a dataset from a pandas dataframe of examples and Fields
        Arguments:
        examples pd.DataFrame: DataFrame of examples
            fields {str: Field}: The Fields to use in this tuple. The
            string is a field name, and the Field is the associated field.
        filter_pred (callable or None): use only exanples for which
            filter_pred(example) is true, or use all examples if None.
            Default is None
        """
        self.examples = examples.apply(SeriesExample.fromSeries, args=(fields,), axis=1).tolist()
        if filter_pred is not None:
            self.examples = filter(filter_pred, self.examples)
        self.fields = dict(fields)
        # Unpack field tuples
        for n, f in list(self.fields.items()):
            if isinstance(n, tuple):
                self.fields.update(zip(n, f))
                del self.fields[n]

                
class SeriesExample(Example):
    """Class to convert a pandas Series to an Example"""

    @classmethod
    def fromSeries(cls, data, fields):
        return cls.fromdict(data.to_dict(), fields)

    @classmethod
    def fromdict(cls, data, fields):
        ex = cls()

        for key, field in fields.items():
#             print(key)
#             print(field)
            if key not in data:
                raise ValueError("Specified key {} was not found in "
                "the input data".format(key))
            if field is not None:
                setattr(ex, key, field.preprocess(data[key]))
            else:
                setattr(ex, key, data[key])
        return ex

In [0]:
def tokenizer(text, stop_ws=exl.stops, stemmer=None, str_output=False):

    t = text.replace("-", " ").split(" ")
    t = [w.strip(string.punctuation) for w in t]

    if stop_ws:
        t = [w.lower() for w in t if w not in stop_ws]

    if stemmer:
        t = [stemmer.stem(w) for w in t]

    if t == []:
        return ['<blank>']
    if str_output:
        return ' '.join(t)
    else:
        return t

### Data Loader Testing

In [0]:


TEXT = Field(tokenize=tokenizer, lower=True, include_lengths = True)
LABEL = Field(sequential=False, unk_token=None, 
              dtype=torch.float)

In [0]:
train_ds = DataFrameDataset(train_sample, fields = {'label' : LABEL, 
                                                    'comment_text': TEXT})
valid_ds = DataFrameDataset(val_samples, fields = {'label' : LABEL, 
                                                   'comment_text': TEXT})
test_ds = DataFrameDataset(test_samples, fields = { 'label' : LABEL, 
                                                   'comment_text' : TEXT})

In [0]:
train_ds.fields

In [0]:
valid_ds.fields

In [0]:
TEXT.build_vocab(train_ds, max_size=25000, 
#                  vectors = "glove.840B.300d",
                 vectors = "glove.6B.200d",
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_ds)

In [0]:
print('len(TEXT.vocab)', len(TEXT.vocab))
# print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

In [0]:
print(TEXT.vocab.stoi['hope'])

In [0]:
print(TEXT.vocab.itos[198])

In [0]:
# # Attribution: Adapted from https://towardsdatascience.com/how-to-use-torchtext-for-neural-machine-translation-plus-hack-to-make-it-5x-faster-77f3884d95
# import torch
# from torchtext import data
# import numpy as np

# global max_text_in_batch

# def batch_size_fn(new, count, sofar):
#     "Keep augmenting batch and calculate total number of tokens + padding."
#     global max_text_in_batch
#     if count == 1:
#         max_text_in_batch = 0
#     max_text_in_batch = max(max_text_in_batch, len(new.comment_text) + 2)
#     text_elements = count * max_text_in_batch
#     return text_elements


# class EfficientIterator(data.Iterator):
#     def create_batches(self):
#         if self.train:
#             def pool(d, random_shuffler):
#                 for p in data.batch(d, self.batch_size * 100):
#                     p_batch = data.batch(
#                         sorted(p, key=self.sort_key),
#                         self.batch_size, self.batch_size_fn)
#                     for b in random_shuffler(list(p_batch)):
#                         yield b
#             self.batches = pool(self.data(), self.random_shuffler)
            
#         else:
#             self.batches = []
#             for b in data.batch(self.data(), self.batch_size,
#                                           self.batch_size_fn):
#                 self.batches.append(sorted(b, key=self.sort_key))

In [0]:
# VOCAB_SIZE = len(TEXT.vocab)
BATCH_SIZE = 1300

In [0]:
train_iter, val_iter, test_iter = BucketIterator.splits(
    (train_ds, valid_ds, test_ds), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.comment_text),
    sort_within_batch = True,
    device = device)

In [0]:
device

In [0]:
# train_iter = EfficientIterator(
#     train_ds, batch_size=1300, 
#     device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
#     repeat=False, sort_key= lambda x: len(x.comment_text), batch_size_fn=batch_size_fn, 
#     train=True, shuffle=True)


In [0]:
batch = next(iter(train_iter))
print(batch.label)
print(batch.label.size())
print(batch.comment_text)
print(batch.comment_text[0].size())

In [0]:
val_batch = next(iter(val_iter))
print(val_batch.label)
print(val_batch.label.size())
print(val_batch.comment_text)
print(val_batch.comment_text[0].size())

In [0]:
test_batch = next(iter(test_iter))
print(test_batch.label)
print(test_batch.label.size())
print(test_batch.comment_text)
print(test_batch.comment_text[0].size())

In [0]:
# val_iter = EfficientIterator(
#     valid_ds, batch_size=1300, 
#     device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
#     repeat=False, sort_key= lambda x: len(x.comment_text), batch_size_fn=batch_size_fn, 
#     train=True, shuffle=True)


### Integrate Batch Loader into Model

In [0]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, pad_idx, batch_size=1,
                 embed_dim=6, weight_decay=0, optimizer_fcn='Adam',
                 learning_rate=1e-3, num_layers=2, dropout=0.05, 
                 num_classes=1, bidirectional=True):
        super(LSTMModel, self).__init__()
        nn.Module.__init__(self)
#         TextData.__init__(self, X_data)
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.embed_dim = embed_dim
        self.dropout = dropout
        self.output_dim = num_classes
        self.loss_fcn = nn.NLLLoss()
        self.weight_decay = weight_decay
        self.learning_rate = learning_rate
        self.pad_idx = pad_idx

        # Layer 1: Embedding Layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_dim, padding_idx=pad_idx)

        # Layer 2: LSTM Layer
        self.lstm = nn.LSTM(input_size = self.embed_dim, 
                            hidden_size = self.hidden_dim,
                            num_layers = self.num_layers, 
                            dropout = self.dropout,
                            bidirectional=bidirectional) 
#                             batch_first=True)

        # Layer 3 (Output Layer): Linear
        self.linear = nn.Linear(self.hidden_dim * 2, self.output_dim)
    
        # Dropout Layer  
        self.dropout_layer = nn.Dropout(self.dropout)

        # define optimizer
        if optimizer_fcn == 'Adam':
            self.optimizer = optim.Adam(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
        elif optimizer_fcn == 'RMSprop':
            self.optimizer = optim.RMSprop(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
        elif optimizer_fcn == 'SDG':
            self.optimizer = optim.SGD(params=self.parameters(),
                                                 weight_decay=self.weight_decay,
                                                 lr=self.learning_rate)
            

    def forward(self, input_seq, input_len):
        # input_seq shape is [batch_max_input_len, batch_size]
        embed_out = self.embedding(input_seq)
        embed_out = self.dropout_layer(embed_out)

        # embed_out shape is [batch_max_input_len, batch_size, embed_dim]

        
        # tell LSTM layer to focus on non-padded elements by packing embeddings
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embed_out, input_len)
        
        packed_lstm_out, (hn, cn) = self.lstm(packed_embedded)
        # hn shape is [num_layers * num_directions, batch_size, hidden_dim]
        
        # unpack sequences into tensors if need them
        # out, out_len = nn.utils.rnn.pad_packed_sequence(packed_lstm_out)
        # out shape is [batch_max_input_len, batch_size, hidden_dim * num directions]
        
        #concat final forward and backward hidden layers to apply any dropout
        hn = self.dropout_layer(torch.cat((hn[-2,:,:], hn[-1,:,:]), dim = 1))
        # hn shape is [batch size, hid dim * num directions]
        
#         out = F.log_softmax(self.linear(hn.squeeze(0)), dim=self.output_dim) 
        out = F.log_softmax(self.linear(hn), dim=1)
        # out shape is [batch_size, outpu_dim]
        

        return out


In [0]:
def train_clf(model, iterator):
    running_loss = 0
    epoch_recall = 0
    epoch_nontarget_recall = 0
    
    model.train()

    for batch in iterator:
#         if len(batch) != iterator.batch_size:
#             print(f"Skipping small batch of length {len(batch)}...")
#             continue
          
        model.optimizer.zero_grad()

        seq, seq_lengths = batch.comment_text
        y = batch.label.long()

        if torch.cuda.is_available():
            seq = seq.cuda() 
            y = y.cuda()
            seq_lengths = seq_lengths.cuda()

        preds = model.forward(seq, seq_lengths)

        loss = model.loss_fcn(input=preds.squeeze(1), 
                                     target=y)

        y_true = [single_tensor.item() for single_tensor in y]

        indices = torch.argmax(preds, 1)
        y_pred = [single_tensor.item() for single_tensor in indices.long()]

        recall_toxic = recall_score(y_true, y_pred, pos_label=1)
        recall_nontoxic = recall_score(y_true, y_pred, pos_label=0)

        loss.backward()

        running_loss += loss.item()
        epoch_recall += recall_toxic.item()
        epoch_nontarget_recall += recall_nontoxic.item()

        model.optimizer.step()
        
    last_mean_error = np.mean(np.abs(loss.item()))
    return running_loss / len(iterator), epoch_recall / len(iterator),  epoch_nontarget_recall / len(iterator), last_mean_error


In [0]:
def evaluate_clf(model, iterator):
    
    running_loss = 0
    epoch_recall = 0
    epoch_nontarget_recall = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            seq, seq_lengths = batch.comment_text
            
            y = batch.label.long()
            
            predictions = model.forward(seq, seq_lengths)
            
            loss = model.loss_fcn(predictions.squeeze(1), y)
            
            val_true = [single_tensor.item() for single_tensor in y]

            indices = torch.argmax(predictions, 1)
            val_pred = [single_tensor.item() for single_tensor in indices]

            f1_toxic = f1_score(val_true, val_pred, pos_label=1)
            f1_nontoxic = f1_score(val_true, val_pred, pos_label=0)
        
            recall_toxic = recall_score(val_true, val_pred, pos_label=1)
            recall_nontoxic = recall_score(val_true, val_pred, pos_label=0)
            
            precision_toxic = precision_score(val_true, val_pred, pos_label=1)
            precision_nontoxic = precision_score(val_true, val_pred, pos_label=0)
            
            accuracy = accuracy_score(val_true, val_pred)
            
            try:
                auc_roc = roc_auc_score(val_true, val_pred)
            except:
                auc_roc = -1

            running_loss += loss.item()
            epoch_recall += recall_toxic.item()
            epoch_nontarget_recall += recall_nontoxic.item()
  
            metrics = {
                "target": {"f1": round(f1_toxic * 100, 5),
                           "precision": round(precision_toxic * 100, 5),
                           "recall": round(recall_toxic * 100, 5)
                          },
                "nontarget": {"f1": round(f1_nontoxic * 100, 5),
                           "precision": round(precision_nontoxic * 100, 5),
                           "recall": round(recall_nontoxic * 100)
                          },
                "overall": {"accuracy": round(accuracy * 100, 5),
                           "auc_roc": round(auc_roc * 100, 5)}
            }
   
        
    return running_loss / len(iterator), epoch_recall / len(iterator),  epoch_nontarget_recall / len(iterator), metrics

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def loop_epochs(model, train_iter, val_iter, num_epochs, outfile=None, 
                model_prefix=None):  
    loop_start = time.time()
    if outfile:
        f = open(outfile, "a")
    else:
        f = None
      
    # best_valid_loss = float('inf')
    last_mean_error = -math.inf
    best_target_recall = -1
    best_nontarget_recall = -1

    thresholds= {
            5000: 1000,
            1000: 100,
            100: 10,
            20: 5,
            10: 3
        }
    metric_results = dict()

    for epoch in range(num_epochs):

        start_time = time.time()

        print(f"Epoch {epoch}:\nStarting training...", file=f)
        train_loss, target_recall, nontarget_recall, epoch_mean_error = train_clf(model, train_iter)

        end_train = time.time()
        train_mins, train_secs = epoch_time(start_time, end_train)

        print(f"Starting evaluation...({train_mins:02}:{train_secs:02} elapsed)", file=f)
        valid_loss, val_target_recall, val_nontarget_recall, metrics = evaluate_clf(model, val_iter)

        end_eval = time.time()
        eval_mins, eval_secs = epoch_time(end_train, end_eval)
        print(f"Finished evaluation...({eval_mins:02}:{eval_secs:02} elapsed)", file=f)

        epoch_mins, epoch_secs = epoch_time(start_time, end_eval)

        metric_results[epoch] = metrics

        if val_target_recall > best_target_recall:
            best_target_recall = val_target_recall
            savestate_file = f'{model_prefix}_epoch{epoch}_target.pt'
            torch.save(model.state_dict(), savestate_file)
            
        if val_nontarget_recall > best_nontarget_recall:
            best_nontarget_recall = val_nontarget_recall
            savestate_file = f'{model_prefix}_epoch{epoch}_nontarget.pt'
            torch.save(model.state_dict(), savestate_file)     

        print(f'Epoch: {epoch+1:02} | Total Epoch Time: {epoch_mins}m {epoch_secs}s', file=f)
        print(f'\tNegative Log Linear Loss:  Loss: {train_loss:.6f} | Target Recall: {target_recall*100:.2f}% | Nontarget Recall: {nontarget_recall*100:.2f}%', file=f)
        print(f'\tVal. Loss: {valid_loss:.6f} | Target Recall: {val_target_recall*100:.2f}% | Nontarget Recall: {val_nontarget_recall*100:.2f}%', file=f)
        
        
    loop_mins, loop_secs = epoch_time(loop_start, end_eval)
    print(f"Total Time: {loop_mins:02}:{loop_secs:02} elapsed", file=f)
    print('', file=f)
    print(metric_results, file=f)
    print('', file=f)
    print('', file=f)
    f.close()
    if outfile:
        send_to_bucket(filename=outfile, directory="model_results")
    
    send_to_bucket(filename=savestate_file, directory="model_states")


### Test Model Functions on 20% of Dataset

In [0]:
NUM_EPOCHS = 6
hist_lstm = np.zeros(NUM_EPOCHS)
model_name="20_pct_dataset"

In [0]:
lstm_model = LSTMModel(vocab_size = VOCAB_SIZE, hidden_dim=100, num_layers=1, 
#                        embed_dim=300, 
                       embed_dim=200, 
                       batch_size=train_iter.batch_size,
                       dropout=0, num_classes=2, 
                       pad_idx=TEXT.vocab.stoi[TEXT.pad_token])

In [0]:
torch.cuda.is_available()

In [0]:
if torch.cuda.is_available():
    lstm_model = lstm_model.cuda()
    lstm_model.loss_fcn = lstm_model.loss_fcn.cuda()

In [0]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

In [0]:
lstm_model.embedding.weight.data.copy_(pretrained_embeddings)

In [0]:
# let model learn from unknown token, while leaving padding as zeros
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

lstm_model.embedding.weight.data[UNK_IDX] = torch.zeros(lstm_model.embed_dim)
lstm_model.embedding.weight.data[lstm_model.pad_idx] = torch.zeros(lstm_model.embed_dim)

print(lstm_model.embedding.weight.data)

In [0]:
loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
            outfile='20pct_imbalanced.txt', model_prefix='20pct_imbalanced')

## Modeling on Rebalanced Datasets

In [0]:
BATCH_SIZE = 500

In [0]:
def initiate_model(train_df, val_df, batch_size=BATCH_SIZE):
    
    TEXT = Field(tokenize=tokenizer, lower=True, include_lengths = True)
    LABEL = Field(sequential=False, unk_token=None, 
              dtype=torch.float)
    
    train_ds = DataFrameDataset(train_df, fields = {'label' : LABEL, 
                                                        'comment_text': TEXT})
    valid_ds = DataFrameDataset(val_df, fields = {'label' : LABEL, 
                                                       'comment_text': TEXT})
#     test_ds = DataFrameDataset(test_df, fields = { 'label' : LABEL, 
#                                                        'comment_text' : TEXT})
    print("Torchtext Datasets created...")
    TEXT.build_vocab(train_ds, max_size=25000, 
#                  vectors = "glove.840B.300d",
                 vectors = "glove.6B.200d",
                 unk_init = torch.Tensor.normal_)
    LABEL.build_vocab(train_ds)
    
    print("Torchtext vocabulary cerated...")
    
    VOCAB_SIZE = len(TEXT.vocab)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

    train_iter, val_iter = BucketIterator.splits(
    (train_ds, valid_ds), 
    batch_size = batch_size,
    sort_key=lambda x: len(x.comment_text),
    sort_within_batch = True,
    device = device)
    
    print("Torchtext iterators cerated...")
    
    lstm_model = LSTMModel(vocab_size = VOCAB_SIZE, hidden_dim=100, num_layers=1, 
                           embed_dim=200, 
                           batch_size=train_iter.batch_size,
                           dropout=0, num_classes=2, 
                           pad_idx=TEXT.vocab.stoi[TEXT.pad_token])
    
    print("Model created...")
    
    if torch.cuda.is_available():
        lstm_model = lstm_model.cuda()
        lstm_model.loss_fcn = lstm_model.loss_fcn.cuda()
        
    pretrained_embeddings = TEXT.vocab.vectors
    lstm_model.embedding.weight.data.copy_(pretrained_embeddings)
    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    lstm_model.embedding.weight.data[UNK_IDX] = torch.zeros(lstm_model.embed_dim)
    lstm_model.embedding.weight.data[lstm_model.pad_idx] = torch.zeros(lstm_model.embed_dim)
    print("Pre-trained embeddings applied...")
    
    return lstm_model, train_iter, val_iter


In [0]:
rebalance_dict = {0: 35, 1: 50, 2: 60, 3: 65, 4:75, 5: 'random'}
data_proportions = [0.2, 0.4, 0.6, 0.8]

### Rebalanced Toxicity on 20% of Dataset

In [0]:

prepared_35, prepared_50, prepared_60, prepared_65, prepared_75, random_df = exl.rebalance_data(train_set)

In [0]:
test_ratio=0.2

for i, df in enumerate([prepared_35, prepared_50, prepared_60, prepared_65, prepared_75, random_df]):
    model_name=f'20pct_model_{rebalance_dict[i]}toxic'
    print(f'Model: {model_name}')
    val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), random_state=1008 )
    
    print(f"Validation sample_created...")
    
    lstm_model, train_iter, val_iter = initiate_model(df, val_sample_df, batch_size=BATCH_SIZE)
    
    print(f"Beginning training and evaluation loops...")

    loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
            outfile=f'{model_name}.txt', model_prefix=model_name)


### Rebalanced Toxicity on 40%, 60%, 80% of Dataset

In [0]:
NUM_EPOCHS = 6
hist_lstm = np.zeros(NUM_EPOCHS)

In [0]:
for proportion in data_proportions[3:]:
    train_set, val_set, test_set = exl.get_samples(train_full, proportion=proportion, 
                                                  train_test_ratio=(1-test_ratio))
    
    pickle_to_gcs(test_set, f'{proportion*100:.0f}pct_test_set.pkl', 
                  directory="test_sets")
        
    rebalanced_dfs = exl.rebalance_data(train_set)
    
    
    
    for i, df in enumerate(rebalanced_dfs):
        if (rebalance_dict[i] in {35}) and (proportion == 0.8):
            continue
   
        model_name=f'{proportion*100:.0f}pct_model_{rebalance_dict[i]}toxic'
        
        print(f'Model: {model_name}')
        
        pickle_to_gcs(df, f'{model_name}_train.pkl', directory="train_sets")
        
        try:
            val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), 
                                           random_state=1008 )
        except:
            val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), 
                                           random_state=1008, replace = True )
            
        pickle_to_gcs(val_sample_df, f'{model_name}_val.pkl', directory="val_sets")

        print(f"Validation sample_created...")

        lstm_model, train_iter, val_iter = initiate_model(df, val_sample_df, 
                                                          batch_size=BATCH_SIZE)

        print(f"Beginning training and evaluation loops...")

        loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
                outfile=f'{model_name}.txt', model_prefix=model_name)


Retrieving training samples at proportion 0.8
Rebalance Ratio: 0.35, 404283 toxic samples out of 1155095
Rebalance Ratio: 0.5, 577547 toxic samples out of 1155095
Rebalance Ratio: 0.6, 693057 toxic samples out of 1155095
Rebalance Ratio: 0.65, 750811 toxic samples out of 1155095
Rebalance Ratio: 0.75, 866321 toxic samples out of 1155095
Rebalanced dfs created...
Model: 80pct_model_35toxic
Validation sample_created...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/80pct_model_35toxic.txt [Content-Type=text/plain]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///content/80pct_model_35toxic_epoch5_nontarget.pt [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.0 MiB

### Large Dataset Troubhleshooting
80% datasets are crashing RAM, so create dfs, pickle, and run individually

In [0]:

train_set, val_set, test_set = exl.get_samples(train_full, proportion=0.8, 
                                              train_test_ratio=(1-test_ratio))

pickle_to_gcs(test_set, f'80pct_test_set_past35.pkl', 
              directory="test_sets")

rebalanced_dfs = exl.rebalance_data(train_set)

for i, df in enumerate(rebalanced_dfs):
    if i == 0:
        continue
    model_name = f'80pct_model_{rebalance_dict[i]}toxic'
    pickle_to_gcs(df, f'{model_name}_train.pkl', directory="train_sets")
    
    try:
        val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), 
                                           random_state=1008 )
        pickle_to_gcs(val_sample_df, f'{model_name}_val.pkl', directory="val_sets")
        
    except:
        val_sample_df = val_set.sample( n=math.ceil(len(df)*test_ratio), 
                                       random_state=1008, replace = True )
        
        pickle_to_gcs(val_sample_df, f'{model_name}_val.pkl', directory="val_sets")

Retrieving training samples at proportion 0.8


/usr/local/lib/python3.6/dist-packages/gcsfs/core.py:304: ResourceWarning: unclosed <ssl.SSLSocket fd=85, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 57806), raddr=('74.125.142.95', 443)>
  self._singleton[0] = self
/usr/local/lib/python3.6/dist-packages/gcsfs/core.py:304: ResourceWarning: unclosed <ssl.SSLSocket fd=84, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 34352), raddr=('74.125.20.84', 443)>
  self._singleton[0] = self


Rebalance Ratio: 0.35, 404273 toxic samples out of 1155068
Rebalance Ratio: 0.5, 577534 toxic samples out of 1155068
Rebalance Ratio: 0.6, 693040 toxic samples out of 1155068
Rebalance Ratio: 0.65, 750794 toxic samples out of 1155068
Rebalance Ratio: 0.75, 866301 toxic samples out of 1155068
Rebalanced dfs created...


In [0]:
def run_given_dfs(train_pkl, val_pkl, gcs=False):
    if gcs:   
        model_name = train_pkl.replace('_train.pkl', '')
        train_df = load_pickle_from_gcs(train_pkl, directory="train_sets") 
        val_df = load_pickle_from_gcs(val_pkl, directory="val_sets")  
    
    else:
        model_name = train_pkl.replace('_train.pkl', '')
        train_df = pd.read_pickle(train_pkl)
        val_df = pd.read_pickle(val_pkl)
    
    print("Initiating model and iterators...")

    lstm_model, train_iter, val_iter = initiate_model(train_df, val_df, 
                                                      batch_size=BATCH_SIZE)

    print(f"Beginning training and evaluation loops...")

    loop_epochs(lstm_model, train_iter, val_iter, num_epochs=NUM_EPOCHS, 
            outfile=f'{model_name}.txt', model_prefix=model_name)
    
    print("Done!")


In [0]:
run_given_dfs('80pct_model_50toxic_train.pkl', '80pct_model_50toxic_val.pkl')

Initiated model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Copying file:///content/80pct_model_50toxic.txt [Content-Type=text/plain]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///content/80pct_model_50toxic_epoch4_nontarget.pt [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.0 MiB.                                     


In [0]:
run_given_dfs('40pct_model_35toxic_train.pkl', '40pct_model_35toxic_val.pkl', gcs=True)


Initiating model and iterators...
Torchtext Datasets created...


.vector_cache/glove.6B.zip: 862MB [00:32, 26.2MB/s]                           
100%|█████████▉| 398557/400000 [00:27<00:00, 15233.09it/s]

Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...


100%|█████████▉| 398557/400000 [00:40<00:00, 15233.09it/s]

Copying file:///content/40pct_model_35toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/40pct_model_35toxic_epoch4_nontarget.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('40pct_model_65toxic_train.pkl', '40pct_model_65toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/40pct_model_65toxic.txt [Content-Type=text/plain]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///content/40pct_model_65toxic_epoch4_target.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('40pct_model_75toxic_train.pkl', '40pct_model_75toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/40pct_model_75toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/40pct_model_75toxic_epoch3_target.pt [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('60pct_model_35toxic_train.pkl', '60pct_model_35toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/60pct_model_35toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/60pct_model_35toxic_epoch5_nontarget.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('80pct_model_60toxic_train.pkl', '80pct_model_60toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/80pct_model_60toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/80pct_model_60toxic_epoch4_target.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('80pct_model_65toxic_train.pkl', '80pct_model_65toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/80pct_model_65toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/80pct_model_65toxic_epoch4_target.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('80pct_model_75toxic_train.pkl', '80pct_model_75toxic_val.pkl', gcs=True)

Initiating model and iterators...
Torchtext Datasets created...
Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...
Pre-trained embeddings applied...
Beginning training and evaluation loops...
Copying file:///content/80pct_model_75toxic.txt [Content-Type=text/plain]...
/ [1 files][  3.0 KiB/  3.0 KiB]                                                
Operation completed over 1 objects/3.0 KiB.                                      
Copying file:///content/80pct_model_75toxic_epoch5_target.pt [Content-Type=application/octet-stream]...
/ [1 files][ 20.0 MiB/ 20.0 MiB]                                                
Operation completed over 1 objects/20.0 MiB.                                     
Done!


In [0]:
run_given_dfs('80pct_model_randomtoxic_train.pkl', '80pct_model_randomtoxic_val.pkl', gcs=True)

/usr/local/lib/python3.6/dist-packages/gcsfs/core.py:304: ResourceWarning: unclosed <ssl.SSLSocket fd=67, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 43386), raddr=('216.58.197.170', 443)>
  self._singleton[0] = self
/usr/local/lib/python3.6/dist-packages/gcsfs/core.py:304: ResourceWarning: unclosed <ssl.SSLSocket fd=66, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('172.28.0.2', 47590), raddr=('172.217.31.173', 443)>
  self._singleton[0] = self


Initiating model and iterators...
Torchtext Datasets created...


.vector_cache/glove.6B.zip: 862MB [16:46, 857kB/s]                            
100%|█████████▉| 398622/400000 [00:27<00:00, 14588.83it/s]

Torchtext vocabulary cerated...
Torchtext iterators cerated...
Model created...


100%|█████████▉| 398622/400000 [00:40<00:00, 14588.83it/s]

Pre-trained embeddings applied...
Beginning training and evaluation loops...


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Copying file:///content/80pct_model_randomtoxic.txt [Content-Type=text/plain]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      
Copying file:///content/80pct_model_randomtoxic_epoch3_target.pt [Content-Type=application/octet-stream]...
-
Operation completed over 1 objects/20.0 MiB.                                     
Done!


## Compare Results

In [0]:
import json

In [0]:
with open('full_list.json', 'r') as f:
    results_dict = json.load(f)

In [0]:
reorg_results = {(k1, k2, k3):v3 for k1,v1 in results_dict.items() \
                       for k2,v2 in results_dict[k1].items()
                       for k3,v3 in results_dict[k1][k2].items()}

In [0]:
results_df = pd.DataFrame([reorg_results[i] for i in sorted(reorg_results)],
                  index=pd.MultiIndex.from_tuples([i for i in sorted(reorg_results.keys())]))    

In [0]:
results_df = results_df.reset_index().rename({'level_0': 'model', 'level_1': 'epoch', 'level_2': 'group'}, axis=1)

In [52]:
results_df[results_df.recall < 100].sort_values(by=['group', 'recall', 'precision', 'model', 'epoch'], ascending=False).style

,index,model,epoch,group,accuracy,auc_roc,f1,precision,recall
467,467,model_results_80pct_model_75toxic,5,target,nan,nan,97.8947,96.875,98.9362
125,125,model_results_20pct_model_60toxic,5,target,nan,nan,98.8701,98.8701,98.8701
329,329,model_results_60pct_model_65toxic,1,target,nan,nan,98.5755,98.2955,98.8571
335,335,model_results_60pct_model_65toxic,3,target,nan,nan,97.7401,96.648,98.8571
449,449,model_results_80pct_model_65toxic,5,target,nan,nan,97.5207,96.7213,98.3333
440,440,model_results_80pct_model_65toxic,2,target,nan,nan,97.5207,96.7213,98.3333
437,437,model_results_80pct_model_65toxic,1,target,nan,nan,97.5207,96.7213,98.3333
122,122,model_results_20pct_model_60toxic,4,target,nan,nan,98.8636,99.4286,98.3051
332,332,model_results_60pct_model_65toxic,2,target,nan,nan,98.0057,97.7273,98.2857
341,341,model_results_60pct_model_65toxic,5,target,nan,nan,97.4504,96.6292,98.2857


In [34]:
results_df[results_df.model == 'model_results_60pct_model_75toxic']

,index,model,epoch,group,accuracy,auc_roc,f1,precision,recall
342,342,model_results_60pct_model_75toxic,0,nontarget,NaN,NaN,100.0000,100.00000,100.0
343,343,model_results_60pct_model_75toxic,0,overall,100.00000,100.0,NaN,NaN,NaN
344,344,model_results_60pct_model_75toxic,0,target,NaN,NaN,100.0000,100.00000,100.0
345,345,model_results_60pct_model_75toxic,1,nontarget,NaN,NaN,0.0000,0.00000,0.0
346,346,model_results_60pct_model_75toxic,1,overall,90.90909,50.0,NaN,NaN,NaN
347,347,model_results_60pct_model_75toxic,1,target,NaN,NaN,95.2381,90.90909,100.0
348,348,model_results_60pct_model_75toxic,2,nontarget,NaN,NaN,0.0000,0.00000,0.0
349,349,model_results_60pct_model_75toxic,2,overall,90.90909,50.0,NaN,NaN,NaN
350,350,model_results_60pct_model_75toxic,2,target,NaN,NaN,95.2381,90.90909,100.0
351,351,model_results_60pct_model_75toxic,3,nontarget,NaN,NaN,0.0000,0.00000,0.0


### Potential Best: 

20% of Data, 75% toxic comments

In [39]:
results_df[results_df.model == '20pct_model_75toxic']

,index,model,epoch,group,accuracy,auc_roc,f1,precision,recall
18,18,20pct_model_75toxic,0,nontarget,NaN,NaN,32.46073,20.00000,86.00000
19,19,20pct_model_75toxic,0,overall,86.78279,86.45981,NaN,NaN,NaN
20,20,20pct_model_75toxic,0,target,NaN,NaN,92.67462,99.39099,86.80851
21,21,20pct_model_75toxic,1,nontarget,NaN,NaN,33.33333,20.66667,86.00000
22,22,20pct_model_75toxic,1,overall,87.29508,86.72577,NaN,NaN,NaN
23,23,20pct_model_75toxic,1,target,NaN,NaN,92.97848,99.39467,87.34043
24,24,20pct_model_75toxic,2,nontarget,NaN,NaN,33.02752,24.65753,50.00000
25,25,20pct_model_75toxic,2,overall,92.52049,72.07447,NaN,NaN,NaN
26,26,20pct_model_75toxic,2,target,NaN,NaN,96.03907,98.00664,94.14894
27,27,20pct_model_75toxic,3,nontarget,NaN,NaN,31.93277,22.89157,53.00000


In [40]:
results_df[results_df.model == 'model_results_80pct_model_35toxic']

,index,model,epoch,group,accuracy,auc_roc,f1,precision,recall
378,378,model_results_80pct_model_35toxic,0,nontarget,NaN,NaN,95.23810,100.00000,91.0
379,379,model_results_80pct_model_35toxic,0,overall,91.42857,95.45455,NaN,NaN,NaN
380,380,model_results_80pct_model_35toxic,0,target,NaN,NaN,57.14286,40.00000,100.0
381,381,model_results_80pct_model_35toxic,1,nontarget,NaN,NaN,96.87500,100.00000,94.0
382,382,model_results_80pct_model_35toxic,1,overall,94.28571,96.96970,NaN,NaN,NaN
383,383,model_results_80pct_model_35toxic,1,target,NaN,NaN,66.66667,50.00000,100.0
384,384,model_results_80pct_model_35toxic,2,nontarget,NaN,NaN,100.00000,100.00000,100.0
385,385,model_results_80pct_model_35toxic,2,overall,100.00000,100.00000,NaN,NaN,NaN
386,386,model_results_80pct_model_35toxic,2,target,NaN,NaN,100.00000,100.00000,100.0
387,387,model_results_80pct_model_35toxic,3,nontarget,NaN,NaN,100.00000,100.00000,100.0


In [45]:
results_df[results_df.model == 'model_results_60pct_model_60toxic']

,index,model,epoch,group,accuracy,auc_roc,f1,precision,recall
306,306,model_results_60pct_model_60toxic,0,nontarget,NaN,NaN,48.88889,39.28571,65.00000
307,307,model_results_60pct_model_60toxic,0,overall,93.42857,79.80039,NaN,NaN,NaN
308,308,model_results_60pct_model_60toxic,0,target,NaN,NaN,96.48855,98.13665,94.89489
309,309,model_results_60pct_model_60toxic,1,nontarget,NaN,NaN,50.00000,53.33333,47.00000
310,310,model_results_60pct_model_60toxic,1,overall,95.42857,72.47836,NaN,NaN,NaN
311,311,model_results_60pct_model_60toxic,1,target,NaN,NaN,97.60479,97.31343,97.89790
312,312,model_results_60pct_model_60toxic,2,nontarget,NaN,NaN,48.48485,50.00000,47.00000
313,313,model_results_60pct_model_60toxic,2,overall,95.14286,72.32821,NaN,NaN,NaN
314,314,model_results_60pct_model_60toxic,2,target,NaN,NaN,97.45127,97.30539,97.59760
315,315,model_results_60pct_model_60toxic,3,nontarget,NaN,NaN,38.70968,42.85714,35.00000


In [49]:
results_df[results_df.model == 'model_results_20pct_model_35toxic'] 

,index,model,epoch,group,accuracy,auc_roc,f1,precision,recall
72,72,model_results_20pct_model_35toxic,0,nontarget,NaN,NaN,96.40288,100.0,93.0
73,73,model_results_20pct_model_35toxic,0,overall,93.05556,-100.0,NaN,NaN,NaN
74,74,model_results_20pct_model_35toxic,0,target,NaN,NaN,0.00000,0.0,0.0
75,75,model_results_20pct_model_35toxic,1,nontarget,NaN,NaN,98.59155,100.0,97.0
76,76,model_results_20pct_model_35toxic,1,overall,97.22222,-100.0,NaN,NaN,NaN
77,77,model_results_20pct_model_35toxic,1,target,NaN,NaN,0.00000,0.0,0.0
78,78,model_results_20pct_model_35toxic,2,nontarget,NaN,NaN,99.30070,100.0,99.0
79,79,model_results_20pct_model_35toxic,2,overall,98.61111,-100.0,NaN,NaN,NaN
80,80,model_results_20pct_model_35toxic,2,target,NaN,NaN,0.00000,0.0,0.0
81,81,model_results_20pct_model_35toxic,3,nontarget,NaN,NaN,99.30070,100.0,99.0


In [51]:
results_df[results_df.model == 'model_results_80pct_model_75toxic']  


,index,model,epoch,group,accuracy,auc_roc,f1,precision,recall
450,450,model_results_80pct_model_75toxic,0,nontarget,NaN,NaN,44.44444,50.00000,40.00000
451,451,model_results_80pct_model_75toxic,0,overall,94.94949,68.93617,NaN,NaN,NaN
452,452,model_results_80pct_model_75toxic,0,target,NaN,NaN,97.35450,96.84211,97.87234
453,453,model_results_80pct_model_75toxic,1,nontarget,NaN,NaN,40.00000,40.00000,40.00000
454,454,model_results_80pct_model_75toxic,1,overall,93.93939,68.40426,NaN,NaN,NaN
455,455,model_results_80pct_model_75toxic,1,target,NaN,NaN,96.80851,96.80851,96.80851
456,456,model_results_80pct_model_75toxic,2,nontarget,NaN,NaN,57.14286,100.00000,40.00000
457,457,model_results_80pct_model_75toxic,2,overall,96.96970,70.00000,NaN,NaN,NaN
458,458,model_results_80pct_model_75toxic,2,target,NaN,NaN,98.42932,96.90722,100.00000
459,459,model_results_80pct_model_75toxic,3,nontarget,NaN,NaN,40.00000,40.00000,40.00000


In [50]:
results_df[results_df.model == 'model_results_20pct_model_60toxic']  

,index,model,epoch,group,accuracy,auc_roc,f1,precision,recall
108,108,model_results_20pct_model_60toxic,0,nontarget,NaN,NaN,27.27273,15.78947,100.00000
109,109,model_results_20pct_model_60toxic,0,overall,91.11111,95.48023,NaN,NaN,NaN
110,110,model_results_20pct_model_60toxic,0,target,NaN,NaN,95.26627,100.00000,90.96045
111,111,model_results_20pct_model_60toxic,1,nontarget,NaN,NaN,30.76923,20.00000,67.00000
112,112,model_results_20pct_model_60toxic,1,overall,95.00000,81.07345,NaN,NaN,NaN
113,113,model_results_20pct_model_60toxic,1,target,NaN,NaN,97.40634,99.41176,95.48023
114,114,model_results_20pct_model_60toxic,2,nontarget,NaN,NaN,33.33333,22.22222,67.00000
115,115,model_results_20pct_model_60toxic,2,overall,95.55556,81.35593,NaN,NaN,NaN
116,116,model_results_20pct_model_60toxic,2,target,NaN,NaN,97.70115,99.41520,96.04520
117,117,model_results_20pct_model_60toxic,3,nontarget,NaN,NaN,25.00000,20.00000,33.00000
